In [ ]:
# K-Nearest Neighbors (KNN)
# Support Vector Machines (SVM)
# Random Forest
# CNN
# Fine-Tuned ResNet-18

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

transform = transforms.Compose([transforms.ToTensor()])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)

print(f"Train samples: {len(trainset)}, Test samples: {len(testset)}")


Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:00<00:00, 37397007.07it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 1044111.21it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:00<00:00, 9952168.68it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 3329929.87it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Train samples: 60000, Test samples: 10000


In [ ]:
# K-Nearest Neighbors (KNN)
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

mnist = fetch_openml('mnist_784', version=1)
X, y = mnist.data, mnist.target.astype(int)

X /= 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"KNN Accuracy on MNIST: {accuracy:.4f}")


KNN Accuracy on MNIST: 0.9713


In [ ]:
# Support Vector Machines (SVM) with PCA (faster)
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

svm_pca = SVC(kernel='rbf', C=10, gamma='scale')
svm_pca.fit(X_train_pca[:5000], y_train[:5000])

y_pred_pca = svm_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
print(f"SVM Accuracy with PCA: {accuracy_pca:.4f}")

SVM Accuracy with PCA: 0.9630


In [4]:
# Random Forest
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Random Forest Accuracy on MNIST: {accuracy_rf:.4f}")

Random Forest Accuracy on MNIST: 0.9669


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
test_loader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 200 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1}/{num_epochs} Completed. Avg Loss: {running_loss/len(train_loader):.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'CNN Accuracy on MNIST: {accuracy:.4f}')


Epoch [1/10], Step [0/938], Loss: 2.3152
Epoch [1/10], Step [200/938], Loss: 0.1443
Epoch [1/10], Step [400/938], Loss: 0.1869
Epoch [1/10], Step [600/938], Loss: 0.1137
Epoch [1/10], Step [800/938], Loss: 0.1315
Epoch 1/10 Completed. Avg Loss: 0.2227
Epoch [2/10], Step [0/938], Loss: 0.1100
Epoch [2/10], Step [200/938], Loss: 0.3192
Epoch [2/10], Step [400/938], Loss: 0.0182
Epoch [2/10], Step [600/938], Loss: 0.1776
Epoch [2/10], Step [800/938], Loss: 0.1259
Epoch 2/10 Completed. Avg Loss: 0.0832
Epoch [3/10], Step [0/938], Loss: 0.1401
Epoch [3/10], Step [200/938], Loss: 0.1029
Epoch [3/10], Step [400/938], Loss: 0.1405
Epoch [3/10], Step [600/938], Loss: 0.0077
Epoch [3/10], Step [800/938], Loss: 0.0524
Epoch 3/10 Completed. Avg Loss: 0.0628
Epoch [4/10], Step [0/938], Loss: 0.0553
Epoch [4/10], Step [200/938], Loss: 0.0369
Epoch [4/10], Step [400/938], Loss: 0.0209
Epoch [4/10], Step [600/938], Loss: 0.0396
Epoch [4/10], Step [800/938], Loss: 0.0417
Epoch 4/10 Completed. Avg Loss:

In [ ]:
#GPU
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader

# transform = transforms.Compose([
#     transforms.ToTensor(), 
#     transforms.Normalize((0.1307,), (0.3081,))
# ])

# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# train_loader = DataLoader(trainset, batch_size=64, shuffle=True)
# test_loader = DataLoader(testset, batch_size=64, shuffle=False)

# class CNN(nn.Module):
#     def __init__(self):
#         super(CNN, self).__init__()
#         self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
#         self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
#         self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
#         self.fc1 = nn.Linear(64 * 7 * 7, 128)
#         self.fc2 = nn.Linear(128, 10)
#         self.dropout = nn.Dropout(0.5)

#     def forward(self, x):
#         x = self.pool(torch.relu(self.conv1(x)))
#         x = self.pool(torch.relu(self.conv2(x)))
#         x = x.view(-1, 64 * 7 * 7)
#         x = torch.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.fc2(x)
#         return x

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = CNN().to(device)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# num_epochs = 10
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for images, labels in train_loader:
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}")

# model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# accuracy = correct / total
# print(f'CNN Accuracy on MNIST: {accuracy:.4f}')


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.models import resnet18

transform = transforms.Compose([
    transforms.Resize((112, 112)),
    transforms.Grayscale(num_output_channels=3),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

trainset = torch.utils.data.Subset(trainset, range(10000))

train_loader = DataLoader(trainset, batch_size=16, shuffle=True, num_workers=2, pin_memory=True)
test_loader = DataLoader(testset, batch_size=16, shuffle=False, num_workers=2, pin_memory=True)

device = torch.device("cpu") 
model = resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False  

model.fc = nn.Linear(512, 10)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

num_epochs = 3  
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 200 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

    print(f"Epoch {epoch+1}/{num_epochs} Completed. Avg Loss: {running_loss/len(train_loader):.4f}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = correct / total
print(f'Fine-Tuned ResNet-18 (Optimized for CPU) Accuracy on MNIST: {accuracy:.4f}')


/Users/ravanryj/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/ravanryj/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch [1/3], Step [0/625], Loss: 2.3484
Epoch [1/3], Step [200/625], Loss: 0.5379
Epoch [1/3], Step [400/625], Loss: 0.4909
Epoch [1/3], Step [600/625], Loss: 0.7005
Epoch 1/3 Completed. Avg Loss: 0.6819
Epoch [2/3], Step [0/625], Loss: 0.3851
Epoch [2/3], Step [200/625], Loss: 0.2921
Epoch [2/3], Step [400/625], Loss: 0.2252
Epoch [2/3], Step [600/625], Loss: 0.3336
Epoch 2/3 Completed. Avg Loss: 0.3799
Epoch [3/3], Step [0/625], Loss: 0.2908
Epoch [3/3], Step [200/625], Loss: 0.1109
Epoch [3/3], Step [400/625], Loss: 0.5412
Epoch [3/3], Step [600/625], Loss: 0.2232
Epoch 3/3 Completed. Avg Loss: 0.3175
Fine-Tuned ResNet-18 (Optimized for CPU) Accuracy on MNIST: 0.9183


In [ ]:
# GPU version
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torchvision
# import torchvision.transforms as transforms
# from torch.utils.data import DataLoader
# from torchvision.models import resnet18

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.Grayscale(num_output_channels=3),
#     transforms.ToTensor(),
#     transforms.Normalize((0.5,), (0.5,))
# ])

# trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
# testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
# test_loader = DataLoader(testset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = resnet18(pretrained=True)

# model.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
# model.fc = nn.Linear(512, 10)

# model = model.to(device)

# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# num_epochs = 5
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0
#     for i, (images, labels) in enumerate(train_loader):
#         images, labels = images.to(device), labels.to(device)

#         optimizer.zero_grad()
#         outputs = model(images)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#         if i % 200 == 0:
#             print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i}/{len(train_loader)}], Loss: {loss.item():.4f}")

#     print(f"Epoch {epoch+1}/{num_epochs} Completed. Avg Loss: {running_loss/len(train_loader):.4f}")

# model.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for images, labels in test_loader:
#         images, labels = images.to(device), labels.to(device)
#         outputs = model(images)
#         _, predicted = torch.max(outputs, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# accuracy = correct / total
# print(f'Fine-Tuned ResNet-18 Accuracy on MNIST: {accuracy:.4f}')
